In [1]:
import pandas as pd 
DATASET = "amazon_reviews_us_Grocery_v1_00.tsv"
import logging
import warnings
import sys
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
logging.disable(sys.maxsize)



C:\Users\geoio\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\geoio\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\geoio\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

C:\Users\geoio\Anaconda3\lib\site-packages\scipy\fft\__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
C:\Users\geoio\Anaconda3\lib\site-packages\scipy\special\orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
C:\Users\geoio\Anaconda3\lib\site-packages\scipy\special\orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias

In [3]:
dataset = pd.read_table(DATASET, error_bad_lines=False, header=0, warn_bad_lines=False) 
# there are some malformed entries the dataset... let's ignore those for now


In [4]:
### Let's see what the dataset looks like... 
dataset.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix,...",Grocery,5,0.0,0.0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nu...",2015-08-31
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0.0,0.0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and ...",2015-08-31
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Ma...,Grocery,5,0.0,0.0,N,N,Five Stars,This green tea tastes so good! My girlfriend l...,2015-08-31
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0.0,0.0,N,Y,Five Stars,I love Melissa's brand but this is a great sec...,2015-08-31
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-...",Grocery,5,0.0,0.0,N,Y,Five Stars,good,2015-08-31


In [5]:
#lots of nan values... :( 
dataset.isnull().sum()

marketplace           0
customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
product_category      0
star_rating          22
helpful_votes        23
total_votes          23
vine                 23
verified_purchase    23
review_headline      34
review_body          88
review_date          65
dtype: int64

In [6]:
# Lots of spammy duplicate reviews...might be worth checking if the same person is duplicating the reviews across multiple products

# Additions, because maybe same review_body is not duplicate i.e. "Good" as review.
x = dataset[dataset['review_body'].duplicated() == True]
x = x[dataset['customer_id'].duplicated() == True] # ADDED THIS
x = x[dataset['product_id'].duplicated() == True] # ADDED THIS
print(len(x))
x.head()

75144


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
2323,US,32433311,R3DPQJFH05T0HS,B00OZYNGUS,228095196,Viva Naturals - The FINEST Raw Organic Chia Se...,Grocery,5,0.0,0.0,N,Y,Great product.,I've been using this to help keep me from snac...,2015-08-31
2330,US,130585,R3IOACXWPYTRUP,B00HZ7HSTW,885171844,Beantown Roasters K Cups Variety Packs,Grocery,5,0.0,1.0,N,Y,The Office like it!,These were purchased for an office and they lo...,2015-08-31
2401,US,14100648,RA8F0DF5OJH3J,B000WV0RW8,653213046,Healthworks Chia Seeds 3lb 6lb Parent,Grocery,5,0.0,0.0,N,Y,GREAT! Thanks So much and God bless,GREAT! Thanks So much and God bless! &#60;&#...,2015-08-31
2436,US,43554935,RK0TC9FRNUL55,B00HQ3ZPJA,220133938,Chia Seeds,Grocery,5,0.0,0.0,N,Y,Five Stars,As advertised.,2015-08-31
2541,US,168472,R2WFNGP0E0ZLXW,B006ZMYLKC,322601427,Keurig,Grocery,5,0.0,0.0,N,Y,Five Stars,Satisfied,2015-08-31


In [7]:
#remove NaNs from the dataset  for now...
dataset = dataset.dropna()

In [8]:
def filter_heuristic(row):
    '''
    determine if a row is a negative review based on its score. Potentially extend it to include sentiment analysis 
    and presence of specific negative words 
    '''
    return row["star_rating"].astype(int) <=3

In [9]:
negative_rows = dataset.loc[filter_heuristic]
negative_rows.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
5,US,23649464,RIG9AWFOGRDVO,B00AL6QBZ6,681475449,Herr's Popcorn Hot Cheese 1 Oz (Pack of 30),Grocery,2,1.0,1.0,N,Y,Not Happy,The popcorn was stale.,2015-08-31
9,US,19624355,R1ODXB3C9UP3NL,B00J074W94,2499702,"Orgain Organic Plant Based Protein Powder, Pac...",Grocery,1,1.0,3.0,N,N,Disgusting now and difficult on digestion,Used to be a decent product. Disgusting now a...,2015-08-31
17,US,22765168,R3T6TTD2IN0EFZ,B00XDXMLL2,971154239,"Skippy Creamy Peanut Butter, with Salted Caram...",Grocery,1,4.0,4.0,N,N,"1 Out Of 5 Of My Co-Workers Thought It Was ""Okay""",I bought this from a local super market on a w...,2015-08-31
23,US,35636887,R9MISLBRG08FX,B00DBSFXUA,294404974,"Keebler Town House Pita Crackers, 9.5 Ounce",Grocery,1,0.0,0.0,N,Y,pita crackers,not craze about these. nothing really wrong wi...,2015-08-31
26,US,12650237,R2A9O8CWZ1PP74,B0083GJKR2,868929824,"Eclipse Sugar Free Gum, Spearmint, 120 Piece B...",Grocery,3,0.0,0.0,N,Y,Three Stars,it's gum..,2015-08-31


In [10]:
sorted_negative_rows = negative_rows.sort_values("product_id")
sorted_negative_rows.head() # ADDITION JUST TO CHECK

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
1975011,US,14434517,R25ATZCV5FBJMV,0805470867,518591127,Communion Bread Pack of 500,Grocery,2,3.0,4.0,N,Y,The only thing my church has ever complained a...,I'm the pastor of a church plant and the only ...,2012-10-16
1365978,US,47145,RUI9CRY6RZNRB,0805470867,518591127,Communion Bread Pack of 500,Grocery,1.0,0.0,0.0,N,Y,I HATE IT,what is that thing<br />it doesn't crunch like...,2014-02-21
1961839,US,37403535,R37YYRMOE7K2TB,0805470867,518591127,Communion Bread Pack of 500,Grocery,1,1.0,1.0,N,Y,Communion bread,"I purchased 3 boxes, two of them the experatio...",2012-11-11
78777,US,41394716,R2QAW3WQRYVQF5,0805470867,518591127,Communion Bread Pack of 500,Grocery,3,0.0,0.0,N,Y,Not what expected,"This came in a flimsy cardboard box, squished....",2015-08-04
1845770,US,11312642,R3M9ORU1I9L2C3,0805470867,518591127,Communion Bread Pack of 500,Grocery,2,0.0,0.0,N,Y,Gross,These things taste nothing like the communion ...,2013-02-18


In [11]:
# def generate_wordcloud(text):
#     wordcloud = WordCloud(width = 5000, height = 5000, random_state=1, background_color='salmon', colormap='Pastel1',
#                           collocations=False, stopwords = STOPWORDS).generate(text)
#     plt.imshow(wordcloud)

In [12]:
# generate a naive word cloud without any preprocessing....
# generate_wordcloud(" ".join(sorted_negative_rows["review_body"]))

In [13]:
dataset.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [14]:
reviews = dataset["review_headline"]
print(reviews.head())
print(sorted_negative_rows["review_headline"])

0    Using these for years - love them.
1                             Wonderful
2                            Five Stars
3                            Five Stars
4                            Five Stars
Name: review_headline, dtype: object
1975011    The only thing my church has ever complained a...
1365978                                            I HATE IT
1961839                                      Communion bread
78777                                      Not what expected
1845770                                                Gross
                                 ...                        
30387                          i hv to say I LOVE THIS CHIPS
1269                                           not very good
18038      Gross Chemical Taste. Nothing like Pumpkin Pie...
22590                                               One Star
2307691                                       Pretty Rough!!
Name: review_headline, Length: 445115, dtype: object


In [15]:
# if not nltk.find('corpora/wordnet'): # may need to download it if it's not in your pc
#     nltk.download('wordnet')
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize('went')) # Default is noun
print(lemmatizer.lemmatize('went', 'v')) # If "v", it works with verbs.
# If needed later, I will spend some time to find the part of speech of each word, and use lemmatizer accordingly

C:\Users\geoio\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:172: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\geoio\Anaconda3\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/releas

went
go


In [16]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim

C:\Users\geoio\Anaconda3\lib\site-packages\google\protobuf\descriptor.py:48: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from google.protobuf.pyext import _message
C:\Users\geoio\Anaconda3\lib\site-packages\gensim\corpora\dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict
C:\Users\geoio\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\geoio\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:73: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated 

In [17]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if (token not in gensim.parsing.preprocessing.STOPWORDS): 
            if (WordNetLemmatizer().lemmatize(token, 'v') != "star"): # Remove star to remove reviews such as: "Five stars"
                result.append(WordNetLemmatizer().lemmatize(token, 'v')) # for now, lemmatizing only verbs
    return result

In [18]:
for review in reviews[:5]:
    print("Removed stopwords and lemmatized words")
    print(review + " becomes -->", preprocess(review))
# Here, I notice that the amount of starts is getting removed. I don't think it's bad because a review being "1 star"
# does not necessarily mean the product is faulty. I can either further remove all the word "stars" from the texts, 
# Or try to figure out how to keep numbers, if it's possible, on lemmatizing

Removed stopwords and lemmatized words
Using these for years - love them. becomes --> ['years', 'love']
Removed stopwords and lemmatized words
Wonderful becomes --> ['wonderful']
Removed stopwords and lemmatized words
Five Stars becomes --> []
Removed stopwords and lemmatized words
Five Stars becomes --> []
Removed stopwords and lemmatized words
Five Stars becomes --> []


In [19]:
reviews_body = dataset["review_body"]
less_reviews_body = reviews_body[:1000]
print(less_reviews_body[10])
processed = []
for review in less_reviews_body:
    processed.append(preprocess(review))
dictionary = gensim.corpora.Dictionary(processed) # construct word<->id mappings - it does it in alphabetical order
#print(dictionary) # In the first 1000 reviews, we have 787 unique words after applying lemmatization in every word
bow_corpus = [dictionary.doc2bow(review) for review in processed]
lda_model_body =  gensim.models.LdaModel(bow_corpus, 
                                   num_topics=20, 
                                   id2word=dictionary,                                    
                                   passes=10,
                                   random_state = 900,
                                   ) 
for idx, topic in lda_model_body.print_topics(-1): # The words occuring in each class, and the weight given for that
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_body, bow_corpus, dictionary)

LDAvis_prepared


I cannot tell the difference in taste between soy sauce and these Bragg Liquid Aminos. I'm glad I bought the small spray bottle of it so that I can have better control of how much I use as it is powerful stuff! Since I'm watching my sodium intake I like to make my own stir-fry dishes and can still enjoy that &#34;soy sauce&#34; flavor enhancer without overdoing it. It's pretty good also on a home cooked burger and beef steak - slather melted butter on top of the cooked meat before serving, give it a quick spray of aminos, a few grinds of black pepper....... and enjoy!
Topic: 0 
Words: 0.026*"love" + 0.017*"use" + 0.016*"buy" + 0.015*"best" + 0.013*"great" + 0.012*"flavor" + 0.011*"taste" + 0.010*"get" + 0.009*"try" + 0.008*"happy"


Topic: 1 
Words: 0.018*"favorite" + 0.017*"taste" + 0.016*"ve" + 0.016*"coffee" + 0.015*"love" + 0.011*"flavor" + 0.010*"ship" + 0.010*"great" + 0.009*"try" + 0.009*"dark"


Topic: 2 
Words: 0.054*"good" + 0.030*"taste" + 0.018*"flavor" + 0.017*"seed" + 0.0

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.037104 -0.062690       1        1  16.555248
5      0.127963 -0.061334       2        1   8.492292
17     0.098540 -0.058237       3        1   6.676447
4      0.021582 -0.078923       4        1   6.194984
11     0.011124  0.018771       5        1   6.046015
3      0.029903 -0.026897       6        1   5.714346
18     0.073663  0.063534       7        1   5.365588
12    -0.139454 -0.115897       8        1   5.293592
0      0.026187 -0.046075       9        1   5.036093
8     -0.013539  0.004924      10        1   4.531425
1      0.059884  0.012994      11        1   4.093443
7     -0.047019 -0.024770      12        1   3.730499
2      0.002394  0.028147      13        1   3.438899
16    -0.122046 -0.081442      14        1   3.337333
6      0.027970  0.082996      15        1   3.306622
19     0.003678  0.007611      16        1   3.032060
9     -0.025098  0.121076      17        1   2.940978
14    -0.041414  0.044745      18        1   2.915452
10    -0.035890  0.094166      19        1   1.983991
13    -0.095532  0.077302      20        1   1.314693, topic_info=      Term        Freq       Total Category  logprob  loglift
152     br  198.000000  198.000000  Default  30.0000  30.0000
18   great  214.000000  214.000000  Default  29.0000  29.0000
21    love  183.000000  183.000000  Default  28.0000  28.0000
44    good  176.000000  176.000000  Default  27.0000  27.0000
47     tea   78.000000   78.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
425   soup    0.643883   11.977460  Topic20  -5.6021   1.4083
185  think    0.643182   31.565627  Topic20  -5.6032   0.4382
257   read    0.643129   11.193134  Topic20  -5.6033   1.4749
75     buy    0.642770  107.035799  Topic20  -5.6038  -0.7836
166   kind    0.642475   15.208309  Topic20  -5.6043   1.1673

[1406 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2319     10  0.621125        abba
1725     17  0.858813     ability
511       8  0.402544    absolute
511      11  0.402544    absolute
249       1  0.241281  absolutely
...     ...       ...         ...
389      12  0.493348       yummy
389      13  0.044850       yummy
389      14  0.089700       yummy
2322     10  0.621124      zabbas
1724      1  0.892120    zucchini

[3162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 6, 18, 5, 12, 4, 19, 13, 1, 9, 2, 8, 3, 17, 7, 20, 10, 15, 11, 14])

In [20]:
example_topic = less_reviews_body[322]
print(example_topic)
bow_vector = dictionary.doc2bow(preprocess(example_topic))
for index, score in sorted(lda_model_body[bow_vector], key=lambda tup: -1*tup[1]):
    print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model_body.print_topic(index, 10)))

The marshmallow bags were difficult to open.  The marshmallows were melted and stuck not only to themselves but the bags.  These were meant to be added to a snack mix for a child's birthday party.  I was unable to use them because they were one/a few big globs of marshmallow and because of that they did not resemble ice cream cones nor did they work for the snack mix.  Very disappointed.
Index: 14
Score: 0.967239260673523	 Topic: 0.019*"love" + 0.010*"good" + 0.010*"add" + 0.010*"nice" + 0.010*"pasta" + 0.010*"tea" + 0.009*"best" + 0.008*"snack" + 0.007*"health" + 0.007*"healthy"


In [21]:
### I will run this on the first 1000 reviews because for now, because it will take long for more
less_reviews = reviews[:1000]
processed = []
for review in less_reviews:
    processed.append(preprocess(review))
print(processed) # All the words after applying lemmatization on the first 1000 reviews

[['years', 'love'], ['wonderful'], [], [], [], ['happy'], [], [], ['great', 'taste'], ['disgust', 'difficult', 'digestion'], ['like', 'soy', 'sauce', 'll', 'like'], [], [], [], [], ['excellent'], ['yummy'], ['workers', 'think', 'okay'], ['delicious', 'tea', 'easy'], ['best', 'coconut', 'oil'], [], [], ['good', 'variety'], ['pita', 'crackers'], [], ['nice', 'big', 'bottle'], [], [], [], [], ['love'], [], ['fresh', 'great', 'tomato', 'base', 'meal'], [], ['goodie', 'downunder'], [], ['far', 'like'], [], ['instructions', 'sure'], ['lifesavers', 'eons', 'trust', 'quality', 'good', 'taste', 'mint'], ['recommend'], [], [], [], ['wonderful', 'watermelon'], ['great', 'product'], ['good', 'flavor'], ['tasteless'], ['tea', 'great', 'enjoy', 'flavor'], ['favorite', 'tea'], ['good'], ['taste', 'great'], ['organic', 'brand', 'better', 'taste'], ['great', 'import', 'area'], [], [], [], [], [], [], ['old', 'stand', 'delicious'], ['yummy'], ['satisfy'], [], [], ['good', 'coconut', 'oil'], [], [], ['fa

In [22]:
dictionary = gensim.corpora.Dictionary(processed) # construct word<->id mappings - it does it in alphabetical order
print(dictionary) # In the first 1000 reviews, we have 787 unique words after applying lemmatization in every word

Dictionary(786 unique tokens: ['love', 'years', 'wonderful', 'happy', 'great']...)


In [23]:
bow_corpus = [dictionary.doc2bow(review) for review in processed]
# Each entry in bow_corpus lists how many times each word id exists. 

print(bow_corpus) # bow_corpus shows the id mapping for each word, and the total occurences in the dict of that word
# in that document

[[(0, 1), (1, 1)], [(2, 1)], [], [], [], [(3, 1)], [], [], [(4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1)], [(9, 2), (10, 1), (11, 1), (12, 1)], [], [], [], [], [(13, 1)], [(14, 1)], [(15, 1), (16, 1), (17, 1)], [(18, 1), (19, 1), (20, 1)], [(21, 1), (22, 1), (23, 1)], [], [], [(24, 1), (25, 1)], [(26, 1), (27, 1)], [], [(28, 1), (29, 1), (30, 1)], [], [], [], [], [(0, 1)], [], [(4, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [], [(35, 1), (36, 1)], [], [(9, 1), (37, 1)], [], [(38, 1), (39, 1)], [(5, 1), (24, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)], [(45, 1)], [], [], [], [(2, 1), (46, 1)], [(4, 1), (47, 1)], [(24, 1), (48, 1)], [(49, 1)], [(4, 1), (20, 1), (48, 1), (50, 1)], [(20, 1), (51, 1)], [(24, 1)], [(4, 1), (5, 1)], [(5, 1), (52, 1), (53, 1), (54, 1)], [(4, 1), (55, 1), (56, 1)], [], [], [], [], [], [], [(18, 1), (57, 1), (58, 1)], [(14, 1)], [(59, 1)], [], [], [(22, 1), (23, 1), (24, 1)], [], [], [(60, 1), (61, 1), (62, 1)], [(63, 1), (64, 1)], [(20, 1), (65, 1), (66, 1), (67

In [24]:
# By changing review_num, we can view the amount of appearances of that word in that specific review
review_num = 0
bow_doc_x = bow_corpus[review_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time(s).".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 0 ("love") appears 1 time(s).
Word 1 ("years") appears 1 time(s).


In [25]:
# Here, we give the bag of words we have (for now I have used first 1000 reviews), how many topics we want to get,
# the dictionary of the id <-> word mappings and how many passes(epochs) to perform, and it creates the model.
lda_model =  gensim.models.LdaModel(bow_corpus, 
                                   num_topics=20, 
                                   id2word=dictionary,                                    
                                   passes=10,
                                   random_state = 900,
                                   ) 

In [26]:
for idx, topic in lda_model.print_topics(-1): # The words occuring in each class, and the weight given for that
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

# I have noticed that every time I run this, the results below are different. This can either be good or bad.
# Some times when I run it, there is a class with the most weighted word being "dissapoint". If we want results that
# are the same every time, we can use a seed for reproducibility. Let me know! 

Topic: 0 
Words: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work" + 0.021*"popcorn" + 0.011*"sage" + 0.011*"thyme" + 0.011*"think" + 0.011*"stick"


Topic: 1 
Words: 0.056*"good" + 0.040*"pretty" + 0.031*"flavor" + 0.020*"pack" + 0.016*"product" + 0.014*"need" + 0.007*"like" + 0.007*"terrific" + 0.007*"buy" + 0.007*"arrive"


Topic: 2 
Words: 0.281*"good" + 0.074*"tea" + 0.046*"taste" + 0.028*"stuff" + 0.024*"enjoy" + 0.024*"better" + 0.018*"yum" + 0.018*"bar" + 0.013*"way" + 0.012*"jif"


Topic: 3 
Words: 0.054*"great" + 0.033*"price" + 0.031*"quality" + 0.027*"flavor" + 0.026*"better" + 0.026*"time" + 0.026*"wonderful" + 0.015*"tea" + 0.013*"use" + 0.013*"average"


Topic: 4 
Words: 0.175*"best" + 0.028*"flavor" + 0.024*"love" + 0.020*"melt" + 0.020*"mix" + 0.019*"expect" + 0.016*"oil" + 0.014*"perfect" + 0.013*"try" + 0.013*"come"


Topic: 5 
Words: 0.081*"flavor" + 0.025*"day" + 0.021*"make" + 0.021*"perfect" + 0.017*"tasteless" + 0.017*"water" + 0

In [27]:
# Here, and on the next cell, I run this on a review that was not used for training to find out how we can get results
# on unseen data
unseen_document = reviews[1001]
print(unseen_document)
preprocess(unseen_document)

Kirkland Coffee is a terrific daily brand coffee.


['kirkland', 'coffee', 'terrific', 'daily', 'brand', 'coffee']

In [28]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

In [29]:
lda_model[bow_vector] # How likely each class is. Because the review is small, some classes have very low probabilities.
# I think that is good! Because there is a high probability to exist in the rest. But again, the data I have trained
# is coarse for now, so no guaranteed results (yet).

[(0, 0.010000195),
 (1, 0.010000195),
 (2, 0.010000195),
 (3, 0.010000195),
 (4, 0.010000195),
 (5, 0.010000195),
 (6, 0.010000195),
 (7, 0.010000195),
 (8, 0.010000195),
 (9, 0.010000195),
 (10, 0.010000197),
 (11, 0.010000195),
 (12, 0.010000195),
 (13, 0.010000195),
 (14, 0.2100584),
 (15, 0.609938),
 (16, 0.010000195),
 (17, 0.010000195),
 (18, 0.010000195),
 (19, 0.010000195)]

In [30]:
# Sorts the most probable classes and gives some details on weights, words and probabilities
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model.print_topic(index, 5)))

Index: 15
Score: 0.6099380850791931	 Topic: 0.249*"great" + 0.093*"taste" + 0.042*"coffee" + 0.028*"thank" + 0.026*"fresh"
Index: 14
Score: 0.21005836129188538	 Topic: 0.055*"brand" + 0.040*"nice" + 0.019*"best" + 0.019*"beautiful" + 0.019*"short"
Index: 10
Score: 0.010000196285545826	 Topic: 0.042*"snack" + 0.027*"eat" + 0.021*"unique" + 0.019*"flavor" + 0.017*"look"
Index: 0
Score: 0.010000195354223251	 Topic: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work"
Index: 1
Score: 0.010000195354223251	 Topic: 0.056*"good" + 0.040*"pretty" + 0.031*"flavor" + 0.020*"pack" + 0.016*"product"
Index: 2
Score: 0.010000195354223251	 Topic: 0.281*"good" + 0.074*"tea" + 0.046*"taste" + 0.028*"stuff" + 0.024*"enjoy"
Index: 3
Score: 0.010000195354223251	 Topic: 0.054*"great" + 0.033*"price" + 0.031*"quality" + 0.027*"flavor" + 0.026*"better"
Index: 4
Score: 0.010000195354223251	 Topic: 0.175*"best" + 0.028*"flavor" + 0.024*"love" + 0.020*"melt" + 0.020*"mix"
Index: 5
S

In [31]:
example_topic = less_reviews[322]
print(example_topic)
bow_vector = dictionary.doc2bow(preprocess(example_topic))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model.print_topic(index, 5)))

# for review in less_reviews:
#     bow_vector = dictionary.doc2bow(preprocess(review))
#     if sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1])[0][0] == 0:
#         print(review)


Disappointed
Index: 0
Score: 0.5249990820884705	 Topic: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work"
Index: 1
Score: 0.02500004880130291	 Topic: 0.056*"good" + 0.040*"pretty" + 0.031*"flavor" + 0.020*"pack" + 0.016*"product"
Index: 2
Score: 0.02500004880130291	 Topic: 0.281*"good" + 0.074*"tea" + 0.046*"taste" + 0.028*"stuff" + 0.024*"enjoy"
Index: 3
Score: 0.02500004880130291	 Topic: 0.054*"great" + 0.033*"price" + 0.031*"quality" + 0.027*"flavor" + 0.026*"better"
Index: 4
Score: 0.02500004880130291	 Topic: 0.175*"best" + 0.028*"flavor" + 0.024*"love" + 0.020*"melt" + 0.020*"mix"
Index: 5
Score: 0.02500004880130291	 Topic: 0.081*"flavor" + 0.025*"day" + 0.021*"make" + 0.021*"perfect" + 0.017*"tasteless"
Index: 6
Score: 0.02500004880130291	 Topic: 0.068*"good" + 0.044*"great" + 0.029*"try" + 0.024*"flavor" + 0.024*"value"
Index: 7
Score: 0.02500004880130291	 Topic: 0.092*"yummy" + 0.053*"excellent" + 0.027*"satisfy" + 0.018*"chocolate" + 0.018*"whi

In [32]:
# If you guys like what I have found with Topic Modelling, we can expand this further and make some further visualisations.
# Maybe we can explore the reviews more or make interesting graphs showing what kind of reviews exist or more.
# Let me know if you guys have any ideas! 

In [33]:


# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.221547 -0.042048       1        1  7.801857
16    -0.045624 -0.009435       2        1  6.826818
2      0.021662  0.261715       3        1  6.422221
12    -0.202159 -0.086655       4        1  5.793389
17     0.088291  0.014043       5        1  5.677224
3     -0.057321  0.044819       6        1  5.669306
4      0.126956 -0.120415       7        1  5.639270
13    -0.083197 -0.037358       8        1  5.458688
1      0.018038  0.093498       9        1  5.364791
5      0.028911 -0.046703      10        1  4.627412
10     0.063134 -0.011548      11        1  4.607650
6     -0.036906  0.114763      12        1  4.520145
11     0.032405 -0.001156      13        1  4.244335
19    -0.074813  0.015591      14        1  4.230055
9      0.007241  0.002621      15        1  4.117900
18     0.038821 -0.029125      16        1  4.017454
8      0.057619 -0.056026      17        1  3.986467
7      0.049736 -0.026546      18        1  3.867468
14     0.103941 -0.059841      19        1  3.771463
0      0.084814 -0.020195      20        1  3.356086, topic_info=          Term       Freq      Total Category  logprob  loglift
0         love  38.000000  38.000000  Default  30.0000  30.0000
24        good  52.000000  52.000000  Default  29.0000  29.0000
4        great  84.000000  84.000000  Default  28.0000  28.0000
47     product  24.000000  24.000000  Default  27.0000  27.0000
21        best  25.000000  25.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
476      hubby   0.705105   2.124885  Topic20  -4.5403   2.2913
77         bar   0.705105   5.725706  Topic20  -4.5403   1.3000
18   delicious   0.705105  12.640641  Topic20  -4.5403   0.5081
345      blend   0.705104   2.137693  Topic20  -4.5403   2.2853
268     season   0.705104   2.772868  Topic20  -4.5403   2.0251

[951 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
451      13  0.698529  absolutely
628       2  0.688065        ache
93       14  0.707373    actually
290      17  0.481089         add
290      18  0.481089         add
...     ...       ...         ...
14       11  0.112363       yummy
14       18  0.786538       yummy
322      12  0.473247       zevia
322      16  0.473247       zevia
678       5  0.718144       zingy

[1157 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 17, 3, 13, 18, 4, 5, 14, 2, 6, 11, 7, 12, 20, 10, 19, 9, 8, 15, 1])

Example that Header of review is better than body: <br>
<em>Review head</em>: Dissapoint <br>
<em>Review body</em>: The marshmallow bags were difficult to open.  The marshmallows were melted and stuck not only to themselves but the bags.  These were meant to be added to a snack mix for a child's birthday party.  I was unable to use them because they were one/a few big globs of marshmallow and because of that they did not resemble ice cream cones nor did they work for the snack mix.  Very disappointed. <br>
<em>Topic by running model on head</em>: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work" <br>
<em>Topic by running model on body</em>: Topic: 0.019*"love" + 0.010*"good" + 0.010*"add" + 0.010*"nice" + 0.010*"pasta" + 0.010*"tea" + 0.009*"best" + 0.008*"snack" + 0.007*"health" + 0.007*"healthy" (MAYBE because of add) <br>
By running on body, no bad Topic categories were found!!
